In [1]:
import re
import os
import csv
import copy
import math
import time
import datetime
import tqdm
import logging
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import glob
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Linear, Dropout
import inspect
import matplotlib.patches as mpatches
from collections import OrderedDict
from torch.nn import LayerNorm
from torch.nn.init import xavier_uniform_
from torchmetrics.text.bert import BERTScore
from GPUtil import showUtilization as gpu_usage
from numba import cuda as cuda1
from torch.utils.data.distributed import DistributedSampler, Sampler
from torch.nn.modules.container import ModuleList
from torch.optim.lr_scheduler import *
from torch.optim import *
from bertviz import model_view, head_view
from rouge import Rouge
from torch.nn.modules.transformer import _get_activation_fn, _get_clones
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler, RandomSampler
from torch.nn.modules import TransformerDecoderLayer as ImportedTransformerDecoderLayer
from transformers import *
from nltk.tokenize import word_tokenize
import math
import numpy as np
import re
import matplotlib.pyplot as plt
import pandas as pd
from collections import OrderedDict
from torchmetrics.text.bert import BERTScore
from rouge import Rouge
import copy
import gc

/home/msc2/Jupyter/jupyterenv/lib/python3.8/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/home/msc2/Jupyter/jupyterenv/lib/python3.8/site-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
2023-04-28 15:40:56.121843: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-28 15:40:56.121898: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


In [2]:
path = f'/media/msc2/A6982E8F982E5DD5/Project/MODELS-DO NOT DELETE NO BACKUP/NEW/ORIG DATA'

pre_roberta_baseline = f'{path}/Roberta/baseline_Checkpoints/model_checkpoint.pt'
roberta_baseline = f'{path}/Roberta/baseline_Checkpoints/model_checkpoint_697.pt'
roberta_pgn = f'{path}/Roberta/pgn_Checkpoints/pgn_model_checkpoint_some.pt'

bert_baseline = f'../../model_checkpoint(1)_x.pt'
bert_pgn = f'../../pgn_model_checkpoint.pt'

device  = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [32]:
def get_loss(train_loss, valid_loss, title = None, num_epochs = None):
    plt.figure(figsize = (12,7))
    plt.title(f'Train Loss and Validation Loss of {title} for {num_epochs} epochs', fontsize = 17)
    plt.xlabel('Epochs', fontsize = 14)
    plt.ylabel('Loss', rotation = 40, fontsize = 14)
    plt.plot(train_loss, linewidth = 0.7, color = 'r')
    plt.plot(valid_loss, color = 'k', alpha = 0.9)
    valid_patch = mpatches.Patch(color = 'r', label = 'Training Loss')
    train_patch = mpatches.Patch(color = 'k', label = 'Validation Loss')
    plt.legend(handles = [valid_patch, train_patch], fontsize = 13)
    plt.savefig(f'./{title}_loss.jpg', dpi = 300)
    plt.show()

In [50]:
def load_checkpoints(ckp_path, ckp_pre = None, _for = None, title = None, num_epochs = None):
    tmp_checkpoint = torch.load(ckp_path, map_location = 'cpu')
    
    if _for == 'loss':
        if ckp_pre is not None:
            pre_checkpoint = torch.load(ckp_pre, map_location = 'cpu')
            tr_loss = pre_checkpoint['plot_train_loss']+tmp_checkpoint['plot_train_loss']
            va_loss = pre_checkpoint['plot_valid_loss']+tmp_checkpoint['plot_valid_loss']
            get_loss(tr_loss, va_loss, title = title, num_epochs = num_epochs)
            del tmp_checkpoint, pre_checkpoint
            gc.collect()
            return
        
        get_loss(tmp_checkpoint['plot_train_loss'], tmp_checkpoint['plot_valid_loss'], title = title, num_epochs = num_epochs)
        del tmp_checkpoint
        gc.collect()
        return

In [55]:
# load_checkpoints(ckp_path = roberta_baseline, ckp_pre = pre_roberta_baseline,
#                  _for = 'loss', title = 'RoBERTa Baseline', num_epochs = 697)

# load_checkpoints(ckp_path = roberta_pgn,
#                  _for = 'loss', title = 'RoBERTa Pointer-Generator', num_epochs = 140)

# load_checkpoints(ckp_path = bert_baseline,
#                  _for = 'loss', title = 'BERT Baseline', num_epochs = 104)

# load_checkpoints(ckp_path = bert_pgn,
#                  _for = 'loss', title = 'BERT Pointer-Generator', num_epochs = 8)

In [126]:
def calculate_rouge_scores(df):
    score = []
    scorer = Rouge()
    for i in range(len(df)):
        row = df.loc[i]
        score.append(scorer.get_scores(row['predicted_summary'], row['original_text'], avg = True))
    return score

In [142]:
scorer = BERTScore(model_name_or_path = path_to_models['roberta'])

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file /media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/sans_robert/config.json
Model config RobertaConfig {
  "_name_or_path": "/media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/sans_robert/",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.27.4",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.json
loading file merges.txt
loading fil

In [143]:
def calculate_bert_scores(df):
    score = []
    for i in range(len(df)):
        row = df.loc[i]
        score.append(scorer(row['predicted_summary'], row['ground_summary']))
    return score

In [139]:
path_to_models = {'roberta' : '/media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/sans_robert/',
                  'bert' : '/media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/bert-trained/',
                  'albert' : '/media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/sanskrit_albert_new/',
                  'vanilla_bert' : '/media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/bert-base-uncased/'
                 }

In [32]:
bert_baseline_preds = pd.read_csv('./bert_baseline_predictions.csv')
bert_pgn_preds = pd.read_csv('./bert_pgn_predictions.csv')

In [208]:
bert_baseline_scores = calculate_bert_scores(bert_baseline_preds)
bert_pgn_scores = calculate_bert_scores(bert_pgn_preds)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file /media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/sans_robert/config.json
Model config RobertaConfig {
  "_name_or_path": "/media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/sans_robert/",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.27.4",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.json
loading file merges.txt
loading fil

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file /media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/sans_robert/config.json
Model config RobertaConfig {
  "_name_or_path": "/media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/sans_robert/",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.27.4",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.json
loading file merges.txt
loading fil

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file /media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/sans_robert/config.json
Model config RobertaConfig {
  "_name_or_path": "/media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/sans_robert/",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.27.4",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.json
loading file merges.txt
loading fil

...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file /media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/sans_robert/config.json
Model config RobertaConfig {
  "_name_or_path": "/media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/sans_robert/",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.27.4",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.json
loading file merges.txt
loading fil

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file /media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/sans_robert/config.json
Model config RobertaConfig {
  "_name_or_path": "/media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/sans_robert/",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.27.4",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.json
loading file merges.txt
loading fil

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [209]:
def shorten(scores):
    d = {'precision': 'p', 'recall':'r', 'f1':'f'}
    nl = []
    for score in scores:
        score = dict((d[key], value) for (key, value) in score.items())
        nl.append(score)
    return nl

def bmean(scores):
    avg_scores = {'r': 0.0, 'p': 0.0, 'f': 0.0}
    
    x = len(scores)
    
    for score in scores:
        for i in score:
            avg_scores[i] += score[i]
    
    for i in avg_scores:
            avg_scores[i] /= x
    return avg_scores
    

def mean(scores):
    avg_scores = {'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0},
                  'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
                  'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}
    
    x = len(scores)
    
    for score in scores:
        for i in score:
            for j in score[i]:
                avg_scores[i][j] += score[i][j]
    
    for score in avg_scores:
        for j in avg_scores[score]:
            avg_scores[score][j]/= x
    
    return avg_scores

In [210]:
bmean(shorten(bert_baseline_scores))

{'r': 0.42680294811725616, 'p': 0.6803450226783753, 'f': 0.5225961416959762}

In [211]:
bmean(shorten(bert_pgn_scores))

{'r': 0.4225618839263916, 'p': 0.6303184181451797, 'f': 0.5047717988491058}

In [212]:
mean(calculate_rouge_scores(bert_baseline_preds))

{'rouge-1': {'r': 0.028859751359751364,
  'p': 0.21999999999999997,
  'f': 0.05049728841775978},
 'rouge-2': {'r': 0.01378250905105125,
  'p': 0.12162698412698414,
  'f': 0.02475344755943888},
 'rouge-l': {'r': 0.020594405594405597,
  'p': 0.16277777777777777,
  'f': 0.03609548157293015}}

In [213]:
mean(calculate_rouge_scores(bert_pgn_preds))

{'rouge-1': {'r': 0.016777939042089983,
  'p': 0.11168831168831168,
  'f': 0.029122806563856574},
 'rouge-2': {'r': 0.00788019254769121,
  'p': 0.0696969696969697,
  'f': 0.014083693725425736},
 'rouge-l': {'r': 0.016777939042089983,
  'p': 0.11168831168831168,
  'f': 0.029122806563856574}}

In [33]:
roberta_baseline_preds = pd.read_csv('./roberta_baseline_predictions.csv')
roberta_pgn_preds = pd.read_csv('./roberta_pgn_predictions.csv')

In [224]:
roberta_baseline_scores = calculate_bert_scores(roberta_baseline_preds)
roberta_pgn_scores = calculate_bert_scores(roberta_pgn_preds)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file /media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/sans_robert/config.json
Model config RobertaConfig {
  "_name_or_path": "/media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/sans_robert/",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.27.4",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.json
loading file merges.txt
loading fil

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file /media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/sans_robert/config.json
Model config RobertaConfig {
  "_name_or_path": "/media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/sans_robert/",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.27.4",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.json
loading file merges.txt
loading fil

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file /media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/sans_robert/config.json
Model config RobertaConfig {
  "_name_or_path": "/media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/sans_robert/",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.27.4",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.json
loading file merges.txt
loading fil

...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file /media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/sans_robert/config.json
Model config RobertaConfig {
  "_name_or_path": "/media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/sans_robert/",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.27.4",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.json
loading file merges.txt
loading fil

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file /media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/sans_robert/config.json
Model config RobertaConfig {
  "_name_or_path": "/media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/sans_robert/",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.27.4",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.json
loading file merges.txt
loading fil

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [225]:
bmean(shorten(roberta_baseline_scores))

{'r': 0.6168489217758178, 'p': 0.9144396960735321, 'f': 0.7361211240291595}

In [226]:
bmean(shorten(roberta_pgn_scores))

{'r': 0.5602015733718873, 'p': 0.9490929901599884, 'f': 0.7035485923290252}

In [221]:
mean(calculate_rouge_scores(roberta_baseline_preds))

{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}

In [222]:
mean(calculate_rouge_scores(roberta_pgn_preds))

{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}

In [34]:
bert_baseline_preds

,original_text,predicted_summary,ground_summary
0,[CLS] पराचीनयगात असमाक समाज सतरीणा विशिषट सथान...,ससकता कत – लोपामदराम ।ितः ।,पराचीनयगात असमाक समाज सतरीणा विशिषट सथान वरतत ...
1,[CLS] असमाक दशः भारतवरषम असति अय हि हिमालयात र...,ससकता ।,"असमाक दशः भारतवरषम असति अतर गगा, यमना, गोदावरी..."
2,[CLS] वरषाकाल सखकरः भवति असमिन काल विशषतया कषक...,ससकता ।,वरषाकाल सखकरः भवति त कषिकारय करवनति नभः मघाछनन...
3,[CLS] असमाक दश सरवास नदीष गगा अतिशरषठा परधाना ...,ससकता गगासमरणमातरण पापः शिरः धनोति इति कथयत ...,असमाक दश सरवास नदीष गगा अतिशरषठा परधाना पवितरत...
4,[CLS] सताम आचारः सदाचारः कथयत सजजनाः यानि करमा...,ससकतम जनाः ।,सताम आचारः सदाचारः कथयत ऋषयशच वदनति यानि अनिनद...
5,[CLS] यसमिन दश वय जनमधारण करमः स हि असमाक दशः ...,ससकतयठन गौरवण ।,यसमिन दश वय जनमधारण करमः स हि असमाक दशः जनमभमि...
6,[CLS] अधययनसय सरवथा परिवरतन जातम तथापि यावत छा...,ससकतसयः सय ।,अधययनसय सरवथा परिवरतन जातम तथापि यावत छातरः छा...
7,[CLS] शासनमन अनशासनम अरथात शासनन निरमितानि निय...,ससकतसय चलचितर करतम समरथाः सय कथमपि आतमोननतिम ...,शासनमन अनशासनम अरथात शासनन निरमितानि नियमनि पा...
8,[CLS] यतर विविधानि पसतकानि पठनारथ सगहीतानि भवन...,ससकत पुस्तकालयः,यतर विविधानि पसतकानि पठनारथ सगहीतानि भवनति तत ...
9,[CLS] अय असमाक विदयालयः असति असय भवनानि भवयानि...,ससकत -ा विदवासः नतारः दशसय विशिषटाः विविध...,अय असमाक विदयालयः असति असय परधानाधयापक [UNK] ब...


In [35]:
bert_pgn_preds

,original_text,predicted_summary,ground_summary
0,[CLS] शासनमन अनशासनम अरथात शासनन निरमितानि निय...,ससकतसय चलचितर सय ।,शासनमन अनशासनम अरथात शासनन निरमितानि नियमनि पा...
1,[CLS] असमाक दश सरवास नदीष गगा अतिशरषठा परधाना ...,ससकता ।ममन -न ।,असमाक दश सरवास नदीष गगा अतिशरषठा परधाना पवितरत...
2,[CLS] सताम आचारः सदाचारः कथयत सजजनाः यानि करमा...,ससकतम जनाः ।,सताम आचारः सदाचारः कथयत ऋषयशच वदनति यानि अनिनद...
3,[CLS] असमाक दशः भारतवरषम असति अय हि हिमालयात र...,ससकता ।,"असमाक दशः भारतवरषम असति अतर गगा, यमना, गोदावरी..."
4,[CLS] पराचीनयगात असमाक समाज सतरीणा विशिषट सथान...,ससकता कत ः म ।ितः ।,पराचीनयगात असमाक समाज सतरीणा विशिषट सथान वरतत ...
5,[CLS] वरषाकाल सखकरः भवति असमिन काल विशषतया कषक...,ससकता ।,वरषाकाल सखकरः भवति त कषिकारय करवनति नभः मघाछनन...
6,[CLS] यतर विविधानि पसतकानि पठनारथ सगहीतानि भवन...,ससकत विद्यालीय मःनविड पुस्तकालय,यतर विविधानि पसतकानि पठनारथ सगहीतानि भवनति तत ...
7,[CLS] अय असमाक विदयालयः असति असय भवनानि भवयानि...,ससकत -ा - करीडापरागणम सविसततम हरितदरवाछननम ...,अय असमाक विदयालयः असति असय परधानाधयापक [UNK] ब...
8,[CLS] अधययनसय सरवथा परिवरतन जातम तथापि यावत छा...,ससकतसीः न ।,अधययनसय सरवथा परिवरतन जातम तथापि यावत छातरः छा...
9,[CLS] यसमिन दश वय जनमधारण करमः स हि असमाक दशः ...,"ससकत -ठन ठन झासीशवरी लकषमीबाई, राणापरताप मवाड...",यसमिन दश वय जनमधारण करमः स हि असमाक दशः जनमभमि...


In [36]:
roberta_baseline_preds

,original_text,predicted_summary,ground_summary
0,<s>अयं अस्माकं विद्यालयः अस्ति । अस्य भवनानि भ...,वो प्यिरि पिद्यालये ्मि ्ति्रानमनिनियान्ािर्ञ...,अयं अस्माकं विद्यालयः अस्ति । अस्य प्रधानाध्या...
1,<s>शासनमनु अनुशासनम् अर्थात शासनेन निर्मितानि ...,विर्र्षार् ्षाय्िशेश्िर्माता</s>ि</s>िर्र्र्त्...,शासनमनु अनुशासनम् अर्थात शासनेन निर्मितानि निय...
2,<s>यस्मिन् देशे वयं जन्मधारणं कुर्मः स हि अस्म...,स्याक् पेशेषःसाःय्ये पेर्या ्िन्मासा पिशे</s>ा...,यस्मिन् देशे वयं जन्मधारणं कुर्मः स हि अस्माकं...
3,<s>अस्माकं देशे सर्वासु नदीषु गंगा अतिश्रेष्ठा...,व्मिना पेशे ्वार् ्त् ोग्निरेवीष्यतीय्रानमनिनि...,अस्माकं देशे सर्वासु नदीषु गंगा अतिश्रेष्ठा प्...
4,<s>वर्षाकाल सुखकरः भवति । अस्मिन् काले विशेषतय...,क्षाण्रिंन् ुन </s>ृशृत्त्र्याःृर्म्ति ् ुतार...,वर्षाकाल सुखकरः भवति । ते कृषिकार्य कुर्वन्ति...
5,<s>यत्र विविधानि पुस्तकानि पठनार्थं संगृहीतानि...,प्रीिदादानसभ्र्न्तानि ्न्थि प्स्हातानावावान् ्...,यत्र विविधानि पुस्तकानि पठनार्थं संगृहीतानि भव...
6,<s>अध्ययनस्य सर्वथा परिवर्तन जातम् तथापि यावत्...,सियाया्वक्निवर्तनःात् ातितुत् ात्रा ात्रातिराद...,अध्ययनस्य सर्वथा परिवर्तन जातम् तथापि यावत् छा...
7,<s>अस्माकं देशः भारतवर्षम् अस्ति । अयं हि हिमा...,स्माना पिशे ारत्या ्ति ियाृग् ्रा पुर्र् ाध्र...,"अस्माकं देशः भारतवर्षम् अस्ति । अत्र गंगा, यमु..."
8,<s>प्राचीनयुगात् अस्माकं समाजे स्त्रीणां विशिष...,स्राजीयाक्र्र्याना स्रित्य्रायां पिद्क्ट् पंरा...,प्राचीनयुगात् अस्माकं समाजे स्त्रीणां विशिष्टं...
9,<s>सताम् आचारः सदाचारः कथ्यते । सज्जनाः यानि क...,व्यायार् ादार् ाये िचिाय्मात्र्त्त्रानेर्णिणा...,सताम् आचारः सदाचारः कथ्यते । ऋषयश्च वदन्ति यान...


In [37]:
roberta_pgn_preds

,original_text,predicted_summary,ground_summary
0,<s>प्राचीनयुगात् अस्माकं समाजे स्त्रीणां विशिष...,व्रारात्र्र्र्यारास्र्श्य्याता वित्त्र्स्यार्स...,प्राचीनयुगात् अस्माकं समाजे स्त्रीणां विशिष्टं...
1,<s>वर्षाकाल सुखकरः भवति । अस्मिन् काले विशेषतय...,क्रान्र््र्</s>्त्</s>्शात्त्न्याार्य्य्त</s>्...,वर्षाकाल सुखकरः भवति । ते कृषिकार्य कुर्वन्ति...
2,<s>अध्ययनस्य सर्वथा परिवर्तन जातम् तथापि यावत्...,व्य्या्य्नित्य्ान्यान्तान्यार्य्</s>्र्य्निरार...,अध्ययनस्य सर्वथा परिवर्तन जातम् तथापि यावत् छा...
3,<s>अयं अस्माकं विद्यालयः अस्ति । अस्य भवनानि भ...,व्स्यार्सित्यारा</s>्य्त््या्यार्रियार्््र्या<...,अयं अस्माकं विद्यालयः अस्ति । अस्य प्रधानाध्या...
4,<s>यत्र विविधानि पुस्तकानि पठनार्थं संगृहीतानि...,सियाितातिन्त्द्यिन्त्न्यास्स्तिय्न्त्य्त्य्यान...,यत्र विविधानि पुस्तकानि पठनार्थं संगृहीतानि भव...
5,<s>शासनमनु अनुशासनम् अर्थात शासनेन निर्मितानि ...,व्र्र्र्र्र्यार््र्श्ित्यात्न्तित्त्न्य्</s>्त...,शासनमनु अनुशासनम् अर्थात शासनेन निर्मितानि निय...
6,<s>सताम् आचारः सदाचारः कथ्यते । सज्जनाः यानि क...,व्रायान् क्नान्</s>ायिन</s>्या्य्तानित्त्य्यान...,सताम् आचारः सदाचारः कथ्यते । ऋषयश्च वदन्ति यान...
7,<s>अस्माकं देशे सर्वासु नदीषु गंगा अतिश्रेष्ठा...,कारारासिश्श्यार्रात्र्स्रित्य्श्यिर्यार्र्त्य्...,अस्माकं देशे सर्वासु नदीषु गंगा अतिश्रेष्ठा प्...
8,<s>अस्माकं देशः भारतवर्षम् अस्ति । अयं हि हिमा...,स्रारा पिश् पार्य्र्र्त््राा पा््र्््त्र््ार्य...,"अस्माकं देशः भारतवर्षम् अस्ति । अत्र गंगा, यमु..."
9,<s>यस्मिन् देशे वयं जन्मधारणं कुर्मः स हि अस्म...,स्रातायिश्श्स्य्रा पार्या</s>्ात्यार्स्श्</s>्...,यस्मिन् देशे वयं जन्मधारणं कुर्मः स हि अस्माकं...


In [38]:
bb = bert_baseline_preds.loc[8]

In [39]:
bp = bert_pgn_preds.loc[6]

In [40]:
rb = roberta_baseline_preds.loc[5]

In [41]:
rp = roberta_pgn_preds.loc[4]

In [42]:
def t(x):
    print(x['original_text'])
    print()
    print(x['predicted_summary'])
    print()
    print(x['ground_summary'])

In [43]:
t(bb)

[CLS] यतर विविधानि पसतकानि पठनारथ सगहीतानि भवनति तत सथानम पसतकालयः उचयत ततर हि तरिविधः पसतकालयः वयकतिगतः, विदयालयीयः, सारवजनिकशच वयकतिगतः पसतकालयः अधयापकाना अनयषा बदधिजीविनाम च भवति विदयालयीयः विदयालयसय अगम भवति छातराणाम अधयापकानाम च जञानवरदधनाय विदयालयीयः पसतकालयः भवति अतर शकषणिकानि पसतकानि सगहीतानि भवनति निरधन - छातराणा कत विदयालयीयः पसतकालयः अतयपयोगी भवति सारवजनिकष पसतकालयष बहविधानि पसतकानि भवनति पसतकालयसमपरकात शनः शनः विदयारचिः जागरति समपरति गीतशीलः पसतकालयः अपि असति 

ससकत पुस्तकालयः

यतर विविधानि पसतकानि पठनारथ सगहीतानि भवनति तत सथानम पसतकालयः उचयत ततर हि तरिविधः पसतकालयः वयकतिगतः, विदयालयीयः, सारवजनिकशच छातराणाम  


In [44]:
t(bp)

[CLS] यतर विविधानि पसतकानि पठनारथ सगहीतानि भवनति तत सथानम पसतकालयः उचयत ततर हि तरिविधः पसतकालयः वयकतिगतः, विदयालयीयः, सारवजनिकशच वयकतिगतः पसतकालयः अधयापकाना अनयषा बदधिजीविनाम च भवति विदयालयीयः विदयालयसय अगम भवति छातराणाम अधयापकानाम च जञानवरदधनाय विदयालयीयः पसतकालयः भवति अतर शकषणिकानि पसतकानि सगहीतानि भवनति निरधन - छातराणा कत विदयालयीयः पसतकालयः अतयपयोगी भवति सारवजनिकष पसतकालयष बहविधानि पसतकानि भवनति पसतकालयसमपरकात शनः शनः विदयारचिः जागरति समपरति गीतशीलः पसतकालयः अपि असति  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

ससकत विद्यालीय मःनविड पुस्तकालय

यतर विविधानि पसतकानि पठनारथ सगहीतानि भवनति तत सथानम पसतकालयः उचयत ततर हि तरिविधः पसतकालयः वयकतिगतः, विदयालयीयः, सारवजनिकशच छातराणाम  [PAD]


In [45]:
t(rb)

<s>यत्र विविधानि पुस्तकानि पठनार्थं संगृहीतानि भवन्ति तत् स्थानम् पुस्तकालयः उच्यते । तत्र हि त्रिविधः पुस्तकालयः व्यक्तिगतः, विद्यालयीयः, सार्वजनिकश्च । व्यक्तिगतः पुस्तकालयः अध्यापकानां अन्येषां बुद्धिजीविनाम् च भवति । विद्यालयीयः विद्यालयस्य अंगम् भवति । छात्राणाम् अध्यापकानाम् च ज्ञानवर्द्धनाय विद्यालयीयः पुस्तकालयः भवति । अत्र शैक्षणिकानि पुस्तकानि संगृहीतानि भवन्ति । निर्धन- छात्राणां कृते विद्यालयीयः पुस्तकालयः अत्युपयोगी भवति । सार्वजनिकेषु पुस्तकालयेषु बहुविधानि पुस्तकानि भवन्ति । पुस्तकालयसम्पर्कात् शनैः शनैः विद्यारुचिः जागर्ति । सम्प्रति गीतशीलः पुस्तकालयः अपि अस्ति ।</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [46]:
t(rp)

<s>यत्र विविधानि पुस्तकानि पठनार्थं संगृहीतानि भवन्ति तत् स्थानम् पुस्तकालयः उच्यते । तत्र हि त्रिविधः पुस्तकालयः व्यक्तिगतः, विद्यालयीयः, सार्वजनिकश्च । व्यक्तिगतः पुस्तकालयः अध्यापकानां अन्येषां बुद्धिजीविनाम् च भवति । विद्यालयीयः विद्यालयस्य अंगम् भवति । छात्राणाम् अध्यापकानाम् च ज्ञानवर्द्धनाय विद्यालयीयः पुस्तकालयः भवति । अत्र शैक्षणिकानि पुस्तकानि संगृहीतानि भवन्ति । निर्धन- छात्राणां कृते विद्यालयीयः पुस्तकालयः अत्युपयोगी भवति । सार्वजनिकेषु पुस्तकालयेषु बहुविधानि पुस्तकानि भवन्ति । पुस्तकालयसम्पर्कात् शनैः शनैः विद्यारुचिः जागर्ति । सम्प्रति गीतशीलः पुस्तकालयः अपि अस्ति ।</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>